In [17]:
import os, sys
sys.path.append(os.path.dirname(os.path.abspath(os.getcwd())))
from src.utils.dataset import load_dataset
from src.models import TadGAN, AttentionTadGAN
from src.dataset.interim_dataset import InterimDataset
from src.dataset.interim.msl import MSLProcessor
from mlprimitives import load_primitive
from orion.evaluation.contextual import contextual_f1_score


import numpy as np
import logging
import matplotlib.pyplot as plt

In [ ]:
source = 'NASA'
dataset = 'MSL'
signal = 'M-2'
tadgan_model = TadGAN.load(source, dataset, signal)
attention_tadgan_model = AttentionTadGAN.load(source, dataset, signal)

In [ ]:
fig, axs = plt.subplots(3, 4, figsize=(20, 10), sharey=False, sharex=True)

tadgan_model.epoch_loss = np.array(tadgan_model.epoch_loss)
attention_tadgan_model.epoch_loss = np.array(attention_tadgan_model.epoch_loss)
epoch_loss_names = [
    ['valid_x_wasserstein_loss', 'fake_x_wasserstein_loss', 'partial_gp_loss_x', 'weighted_loss'],
    ['valid_z_wasserstein_loss', 'fake_z_wasserstein_loss', 'partial_gp_loss_z', 'weighted_loss'],
    ['fake_gen_x_wasserstein_loss', 'fake_gen_z_wasserstein_loss', 'mse', 'weighted_loss'],
]

index = [i for i in range(tadgan_model.epoch_loss.shape[0])]
for i in range(tadgan_model.epoch_loss.shape[1]):
    for j in range(tadgan_model.epoch_loss.shape[2]):
        axs[i][j].set_title(epoch_loss_names[i][j])
        axs[i][j].plot(index, tadgan_model.epoch_loss[:, i, j], label='tadgan', color='b')
        axs[i][j].plot(index, attention_tadgan_model.epoch_loss[:, i, j], label='attention_tadgan', color='r')

fig.suptitle(f'[{source}] [{dataset}] [{signal}] Epoch losses', fontsize=16)
handles, labels = axs[0][0].get_legend_handles_labels()
fig.legend(handles, labels, loc='upper right')
plt.show()

In [ ]:
def get_anomalies(y_test, y_hat, critic, index_test):
    params = {"rec_error_type": "dtw", "comb": "mult"}

    primitive = load_primitive("orion.primitives.tadgan.score_anomalies", 
                               arguments=params)
    errors, true_index, true, predictions = primitive.produce(y=y_test, y_hat=y_hat, critic=critic, index=index_test)
    
    params = {
    "window_size_portion": 0.33, 
    "window_step_size_portion": 0.1,
    "fixed_threshold": True
    }

    primitive = load_primitive("orion.primitives.timeseries_anomalies.find_anomalies", 
                               arguments=params)
    predicted_anomalies = primitive.produce(errors=errors, index=true_index)
    predicted_anomalies = [(int(anomaly[0]), int(anomaly[1])) for anomaly in predicted_anomalies]
    return predicted_anomalies
    

In [ ]:
interim_dataset = InterimDataset.load(source, dataset, signal)
processor = MSLProcessor(signal)

X_train, y_train, index_train = processor.fit(interim_dataset.X_train, interim_dataset.index_train)
X_train.shape, y_train.shape, index_train.shape

X_test, y_test, index_test = processor.transform(interim_dataset.X_test, interim_dataset.index_test)
X_test.shape, y_test.shape, index_test.shape

In [ ]:
interim_dataset.anomalies_test.values

In [ ]:
print('Ground Truth', interim_dataset.anomalies_test.values)

y_hat, critic = tadgan_model.predict(X_test, y_test)
predicted_anomalies = get_anomalies(y_test, y_hat, critic, index_test)

f1_score = contextual_f1_score(
    interim_dataset.anomalies_test, 
    predicted_anomalies, 
    start=index_test[0], 
    end=index_test[-1], 
    weighted=True
)
print(tadgan_model.model_name, predicted_anomalies, f1_score)


y_hat, critic = attention_tadgan_model.predict(X_test, y_test)
predicted_anomalies = get_anomalies(y_test, y_hat, critic, index_test)

f1_score = contextual_f1_score(
    interim_dataset.anomalies_test, 
    predicted_anomalies, 
    start=index_test[0], 
    end=index_test[-1], 
    weighted=True
)
print(attention_tadgan_model.model_name, predicted_anomalies, f1_score)
